In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# Install packages here
# Packages for data processing
import numpy as np
import pandas as pd
import datetime
from sklearn import preprocessing
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from scipy.sparse import csr_matrix
import scipy as sp


# Packages for visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Packages for modeling
from surprise import Reader
from surprise import Dataset
from surprise import KNNWithMeans
from surprise import KNNBasic
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
from surprise import SVD
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
import heapq

# Packages for model evaluation
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from time import time

# Package to suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Packages for saving models
import pickle

In [2]:
train_df = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/train.csv')
# movies_df = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/movies.csv')
# imdb_df = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/imdb_data.csv')
test_df = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/test.csv')
# links_df = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/links.csv')
# tags = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/tags.csv')
# genome_scores = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/genome_scores.csv')
# genome_tags = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/genome_tags.csv')
# sample_submissions = pd.read_csv('/kaggle/input/alx-movie-recommendation-project-2024/sample_submission.csv')

In [ ]:
imdb_df.head()

In [ ]:
imdb_df.shape

## EDA


### Outliers
- **Identify outliers**: Outliers are data points that differ significantly from other observations. They can skew and mislead the training process of a machine learning model.
- **Detecting outliers**: Use statistical methods such as Z-scores or IQR (Interquartile Range) to detect outliers.
- **Handling outliers**: Decide whether to remove or transform the outliers depending on their impact on the dataset.

### Understanding Relationships Between Various Attributes and Structure of the Data
- **Correlation Analysis**: Use correlation matrices to understand the relationships between numerical attributes.
- **Visualization Techniques**: Employ scatter plots, pair plots, and heatmaps to visualize and explore relationships.
- **Data Structure**: Understand the structure of the data, including the distribution of values and the presence of any missing values.

### Recognizing Important Variables
- **Feature Importance**: Use techniques like Random Forests, Gradient Boosting, or SHAP values to determine feature importance.
- **Domain Knowledge**: Incorporate domain expertise to identify which variables are likely to be important.
- **Statistical Tests**: Conduct statistical tests to identify variables that have significant effects on the target variable.

By understanding the data through these steps, we ensure a robust foundation for building and evaluating machine learning models.


In [ ]:
print("Train: ")
print(str(train_df.isnull().sum()))
print("************")
print("Test: ")
print(str(test_df.isnull().sum()))
print("************")
print("Movies: ")
print(str(movies_df.isnull().sum()))
print("************")
print("Links: ")
print(str(links_df.isnull().sum()))
print("************")
print("IMDB: ")
print(str(imdb_df.isnull().sum()))
print("************")
print("Genome scores: ")
print(str(genome_scores.isnull().sum()))
print("************")
print("Genome tags: ")
print(str(genome_tags.isnull().sum()))

Data preparation is the process of preparing raw data so that it is suitable for further processing and analysis. Key steps include:

- **Collecting**: Gathering raw data from various sources.
- **Cleaning**: Removing or correcting any errors or inconsistencies in the data. This includes handling missing values, correcting data types, and removing duplicates.
- **Labeling**: Annotating data with labels that are required for supervised machine learning tasks. This involves identifying and marking the target variable.
- **Transforming**: Converting raw data into a format that is suitable for analysis. This includes normalization, scaling, encoding categorical variables, and feature engineering.
- **Exploring**: Analyzing the data to understand its structure and relationships. This step includes generating descriptive statistics and visualizing the data to identify patterns and insights.
- **Visualizing**: Creating graphical representations of the data to better understand distributions, trends, and relationships among variables. Common techniques include histograms, bar charts, scatter plots, and heatmaps.

By following these steps, raw data is transformed into a structured format that is ready for machine learning algorithms and further analysis.


In [ ]:
movies_df.head()

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
# Create dataframe containing only the movieId and genres
movies_genres = pd.DataFrame(movies_df[['movieId', 'genres']],
                             columns=['movieId', 'genres'])

# Split genres seperated by "|" and create a list containing the genres allocated to each movie
movies_genres.genres = movies_genres.genres.apply(lambda x: x.split('|'))

# Create expanded dataframe where each movie-genre combination is in a seperate row
movies_genres = pd.DataFrame([(tup.movieId, d) for tup in movies_genres.itertuples() for d in tup.genres],
                             columns=['movieId', 'genres'])

movies_genres.head()

**Lets plot genres from most common to least common**

In [ ]:
plot = plt.figure(figsize=(15, 10))
plt.title('Most common genres\n', fontsize=20)
sns.countplot(y="genres", data=movies_genres,
              order=movies_genres['genres'].value_counts(ascending=False).index,
              palette='Reds_r')
plt.show()

## Modelling phase
 You only need to apply one version
be it Content based or Collabrative method



In [ ]:
import pandas as pd
from surprise import Reader, Dataset, SVD, accuracy
from surprise.model_selection import train_test_split

# Sample books data
books_data = {
    'bookId': [1, 2, 3, 4, 5, 6, 7],
    'title': ['To Kill a Mockingbird', '1984', 'The Great Gatsby', 'Pride and Prejudice', 'The Catcher in the Rye', 'Animal farm'
              ,'Harry Potter']
}

# Create the books DataFrame
books = pd.DataFrame(books_data)

# Sample user profiles
user_profiles = {
    1: {'bookId': [2, 3, 4], 'rating': [5, 4, 3]},
    2: {'bookId': [1, 3, 5], 'rating': [4, 5, 3]}
}

# Combine user profiles into a single DataFrame
all_ratings = []

for user_id, profile in user_profiles.items():
    for book_id, rating in zip(profile['bookId'], profile['rating']):
        all_ratings.append({'userId': user_id, 'bookId': book_id, 'rating': rating})

ratings_df = pd.DataFrame(all_ratings)

# Define the Reader and Dataset
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings_df[['userId', 'bookId', 'rating']], reader)
data

In [ ]:
# Split the data into training and test sets
trainset, testset = train_test_split(data, test_size=0.2, )

# Train the SVD model
svd = SVD()
svd.fit(trainset)

# Make predictions on the test set
predictions = svd.test(testset)

# Compute and print the RMSE
rmse = accuracy.rmse(predictions)
print(f"RMSE: {rmse}")

pred = pd.DataFrame(predictions)
pred

## Recommendation Function

In [ ]:
# Function to get collaborative recommendations for a user profile
def get_collaborative_recommendations(user_id, svd, books, ratings_df, n=10):
    recommendations = []
    book_ids = books['bookId'].unique()

    for book_id in book_ids:
        prediction = svd.predict(user_id, book_id)
        actual_rating = ratings_df[(ratings_df['userId'] == user_id) & (ratings_df['bookId'] == book_id)]['rating']
        actual_rating = actual_rating.values[0] if not actual_rating.empty else None
        recommendations.append((books[books['bookId'] == book_id]['title'].values[0], prediction.est, actual_rating))

    recommendations = sorted(recommendations, key=lambda x: x[1], reverse=True)

    return recommendations[:n]

# Generate recommendations for each user profile and collect them in a list
all_recommendations = []

for user_id in user_profiles.keys():
    recommendations = get_collaborative_recommendations(user_id, svd, books, ratings_df)
    for title, predicted_rating, actual_rating in recommendations:
        all_recommendations.append({
            'userId': user_id,
            'Recommended Book': title,
            'Predicted_Rating': predicted_rating,
            'Actual Rating': actual_rating
        })

# Convert the list of recommendations to a DataFrame
recommendations_df = pd.DataFrame(all_recommendations)

recommendations_df

# Generate your outputs here

Prepare Submission File
We make submissions in CSV files. Your submissions usually have two columns: an ID column and a prediction column. The ID field comes from the test data (keeping whatever name the ID field had in that data, which for the data is the string 'Id'). The prediction column will use the name of the target field.

We will create a DataFrame with this data, and then use the dataframe's to_csv method to write our submission file. Explicitly include the argument index=False to prevent pandas from adding another column in our csv file.

In [ ]:
# This is an example
my_submission = pd.DataFrame({'id': recommendations_df.userId,'predict': recommendations_df.Predicted_Rating})
#you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)

# Tips
- NB:Read the description well
    - REMEMBER: Concatenated ID
    - Evatualtion metrics
- Sampling is your friend -> start small and scale up
- Data ingestioon, pleasse ensure the correct path is dependant on the environment
- Ensure test output matches dimension of test set for Kaggle submission
- 20 Submissions per day
- This is individual project
- Ensure email correlates to Athena for effective tracking
- If you use a shuffler ensure test output aligns with test sample ordering
  - (from sklearn.utils import shuffle), for randomness
- Make sure your notebook is in the same folder
- Analyse your data well
- Make sure you have gone through your content


# Testing Ideas

In [ ]:
sampled_df = train_df.sample(frac=0.5, random_state=42)
sampled_df.shape

In [ ]:
from surprise import Reader, Dataset, SVD, accuracy
from surprise.model_selection import train_test_split

# Define the Reader object
reader = Reader(rating_scale=(0.5, 5.0))

# Load the dataset
data = Dataset.load_from_df(train_df[['userId', 'movieId', 'rating']], reader)

# Split the data into train and test sets
trainset, testset = train_test_split(data, test_size=0.2)

# Initialize the SVD algorithm
algo = SVD()

# Train the algorithm on the training set
algo.fit(trainset)

# Predict ratings for the test set
predictions = algo.test(testset)

# Calculate RMSE
rmse = accuracy.rmse(predictions)
print(f'RMSE: {rmse}')


In [ ]:
"""
Using sample fraction of 0.05
RMSE: 0.9290
RMSE: 0.9290476877052269

Using sample fraction of 0.25
RMSE: 0.8990
RMSE: 0.8989952813217597

Using sample fraction of 0.05
RMSE: 0.8760
RMSE: 0.8759642544462234

Using the whole dataset from the train_df
RMSE: 0.8347
RMSE: 0.8346884683865706

"""

# Testing the whole dataset from the method above

In [ ]:
from surprise import Reader, Dataset, SVD, accuracy
from surprise.model_selection import train_test_split
import timeit

# This is to start the execution time for the cell
start_time = timeit.default_timer()


# Step 2: Prepare Data
# Define the Reader object
reader = Reader(rating_scale=(0.5, 5.0))

# Load the dataset
data = Dataset.load_from_df(train_df[['userId', 'movieId', 'rating']], reader)

# Step 3: Train-Test Split (for model evaluation, not for actual prediction)
trainset, _ = train_test_split(data, test_size=0.2)

# Initialize the SVD algorithm
algo = SVD()

# Train the algorithm on the trainset
algo.fit(trainset)

# Step 4: Predict Ratings for the Test Set
# Prepare a list of (userId, movieId) pairs from test_df
testset = list(zip(test_df['userId'], test_df['movieId'], [0]*len(test_df)))  # the third element is a dummy rating

# Predict ratings for the testset
predictions = algo.test(testset)

# Extract predictions
predicted_ratings = [pred.est for pred in predictions]

# Add predicted ratings to test_df
test_df['rating'] = predicted_ratings

# Optional: Save the predictions to a new CSV file
test_df.to_csv('predicted_ratings.csv', index=False)

print(test_df.head())

elapsed = timeit.default_timer() - start_time
print(elapsed)

In [ ]:
test_df.shape

In [ ]:
testdf = test_df.copy()

In [ ]:
def combine_columns(row):
    return f"{int(row['userId'])}_{int(row['movieId'])}"

testdf['combined'] = testdf.apply(combine_columns, axis=1)

In [ ]:
testdf

In [ ]:
# Select only the 'combined' and 'rating' columns
result_df = testdf[['combined', 'rating']]
result_df = result_df.rename(columns={'combined': 'Id'})
# Print the resulting DataFrame

result_df.to_csv('predicted_ratings.csv', index=False)

print(result_df)

In [ ]:
result_df.shape

# The above code has given me an RMSE of 0.083 -> Now I want to improve my result by using a hyper parameter tuning

In [ ]:
# Timer start
start_time = timeit.default_timer()
# Data is already loaded

# Prepare Data
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(train_df[['userId', 'movieId', 'rating']], reader)

# Train-Test Split
trainset, testset = train_test_split(data, test_size=0.2)

# Hyperparameter Tuning
param_grid = {
    'n_factors': [50, 100, 150],
    'n_epochs': [20, 30, 40],
    'lr_all': [0.002, 0.005, 0.01],
    'reg_all': [0.02, 0.05, 0.1]
}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)
gs.fit(data)
best_params = gs.best_params['rmse']

# Train the best model
algo = SVD(**best_params)
algo.fit(trainset)

# Predict Ratings for the Test Set
testset = list(zip(test_df['userId'], test_df['movieId'], [0]*len(test_df)))
predictions = algo.test(testset)
predicted_ratings = [pred.est for pred in predictions]

# Add predicted ratings to test_df
test_df['rating'] = predicted_ratings
test_df['combined'] = test_df.apply(lambda row: f"{int(row['userId'])}_{int(row['movieId'])}", axis=1)
result_df = test_df[['combined', 'rating']]

# Save the results to a CSV file
result_df.to_csv('predicted_ratings_2.csv', index=False)
print(result_df.shape)
print(result_df.head())

# Timing the amount of time it takes to complete the process. The one without the grid was 304sec.
elapsed = timeit.default_timer() - start_time
print(elapsed)

# The above code took a very long time so I tried to decrease the time complexity using the below code

In [ ]:
import pandas as pd
from surprise import Reader, Dataset, SVD, accuracy
from surprise.model_selection import train_test_split, GridSearchCV
import timeit

# Timer start
start_time = timeit.default_timer()

# Prepare Data
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(train_df[['userId', 'movieId', 'rating']], reader)

# Sample 20% of the data for hyperparameter tuning
sample_data = train_df.sample(frac=0.3, random_state=42)
sample_data = Dataset.load_from_df(sample_data[['userId', 'movieId', 'rating']], reader)
sample_trainset = sample_data.build_full_trainset()

# Hyperparameter Tuning
param_grid = {
    'n_factors': [50, 100],
    'n_epochs': [20, 30],
    'lr_all': [0.002, 0.005],
    'reg_all': [0.02, 0.05]
}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=2, n_jobs=-1)
gs.fit(sample_data)
best_params = gs.best_params['rmse']

# Train-Test Split on Full Data
trainset, testset = train_test_split(data, test_size=0.2)

# Train the best model
algo = SVD(**best_params)
algo.fit(trainset)

# Predict Ratings for the Test Set
testset = list(zip(test_df['userId'], test_df['movieId'], [0]*len(test_df)))
predictions = algo.test(testset)
predicted_ratings = [pred.est for pred in predictions]

# Add predicted ratings to test_df
test_df['rating'] = predicted_ratings
test_df['combined'] = test_df.apply(lambda row: f"{int(row['userId'])}_{int(row['movieId'])}", axis=1)
result_df = test_df[['combined', 'rating']]

result_df = result_df.rename(columns={'combined': 'Id'})

# Save the results to a CSV file
result_df.to_csv('predicted_ratings_2.csv', index=False)
print(result_df.shape)
print(result_df.head())

# Timing the amount of time it takes to complete the process
elapsed = timeit.default_timer() - start_time
print(elapsed)


In [ ]:
result_df.to_csv('predicted_ratings_2.csv', index=False)

In [ ]:
import pandas as pd
from surprise import Reader, Dataset, SVD, SVDpp, NMF, accuracy
from surprise.model_selection import train_test_split, GridSearchCV, cross_validate
import timeit

# Timer start
start_time = timeit.default_timer()

# Prepare Data
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(train_df[['userId', 'movieId', 'rating']], reader)

# Sample 30% of the data for hyperparameter tuning
sample_data = train_df.sample(frac=1, random_state=42)
sample_data = Dataset.load_from_df(sample_data[['userId', 'movieId', 'rating']], reader)
sample_trainset = sample_data.build_full_trainset()

# Expanded Hyperparameter Tuning
param_grid = {
    'n_factors': [20, 50, 100, 150],
    'n_epochs': [10, 20, 30, 40],
    'lr_all': [0.001, 0.002, 0.005, 0.01],
    'reg_all': [0.01, 0.02, 0.05, 0.1]
}

gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=2, n_jobs=-1)
gs.fit(sample_data)
best_params = gs.best_params['rmse']

# Train-Test Split on Full Data
trainset, testset = train_test_split(data, test_size=0.2)

# Train the best model
algo = SVD(**best_params)
algo.fit(trainset)

# Predict Ratings for the Test Set
testset = list(zip(test_df['userId'], test_df['movieId'], [0]*len(test_df)))
predictions = algo.test(testset)
predicted_ratings = [pred.est for pred in predictions]

# Add predicted ratings to test_df
test_df['rating'] = predicted_ratings
test_df['combined'] = test_df.apply(lambda row: f"{int(row['userId'])}_{int(row['movieId'])}", axis=1)
result_df = test_df[['combined', 'rating']]
result_df = result_df.rename(columns={'combined': 'Id'})

# Save the results to a CSV file
result_df.to_csv('predicted_ratings_4.csv', index=False)
print(result_df.shape)
print(result_df.head())

# Timing the amount of time it takes to complete the process
elapsed = timeit.default_timer() - start_time
print(f"{elapsed} in seconds in Mins = {elapsed / 60} mins")

# Cross-validate SVD
algo = SVD()
cv_results = cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)
print(f"Cross-validated RMSE: {cv_results['test_rmse'].mean()}")

In [ ]:
import pandas as pd
from surprise import Reader, Dataset, SVD, SVDpp, NMF, KNNBasic, accuracy
from surprise.model_selection import train_test_split, cross_validate
import timeit

# Timer start
start_time = timeit.default_timer()

# Prepare Data
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(train_df[['userId', 'movieId', 'rating']], reader)

# Train-Test Split on Full Data
trainset, testset = train_test_split(data, test_size=0.2)

# Define and train SVD model
svd = SVD(n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.02)
svd.fit(trainset)

# Define and train SVD++ model
svdpp = SVDpp(n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.02)
svdpp.fit(trainset)

# Define and train NMF model
nmf = NMF(n_factors=100, n_epochs=20, reg_pu=0.02, reg_qi=0.02)
nmf.fit(trainset)

# Define and train KNNBasic model
sim_options = {'name': 'cosine', 'user_based': False}
knn = KNNBasic(sim_options=sim_options)
knn.fit(trainset)

# Predict Ratings for the Test Set
testset = list(zip(test_df['userId'], test_df['movieId'], [0]*len(test_df)))

predictions_svd = svd.test(testset)
predictions_svdpp = svdpp.test(testset)
predictions_nmf = nmf.test(testset)
predictions_knn = knn.test(testset)

# Blend the predictions (simple average)
blended_predictions = []
for svd_pred, svdpp_pred, nmf_pred, knn_pred in zip(predictions_svd, predictions_svdpp, predictions_nmf, predictions_knn):
    blended_est = (svd_pred.est + svdpp_pred.est + nmf_pred.est + knn_pred.est) / 4
    blended_predictions.append((svd_pred.uid, svd_pred.iid, blended_est))

# Calculate RMSE for blended model
test_df['rating'] = [pred[2] for pred in blended_predictions]
predicted_ratings = [pred[2] for pred in blended_predictions]

# Add predicted ratings to test_df
test_df['rating'] = predicted_ratings
test_df['combined'] = test_df.apply(lambda row: f"{int(row['userId'])}_{int(row['movieId'])}", axis=1)
result_df = test_df[['combined', 'rating']]
result_df = result_df.rename(columns={'combined': 'Id'})

# Save the results to a CSV file
result_df.to_csv('predicted_ratings_5.csv', index=False)
print(result_df.shape)
print(result_df.head())

# Timing the amount of time it takes to complete the process
elapsed = timeit.default_timer() - start_time
print(f"Elapsed time: {elapsed} seconds")

# Evaluate the RMSE of the blended model
blended_rmse = accuracy.rmse([pred[2] for pred in blended_predictions])
print(f"Blended Model RMSE: {blended_rmse}")

# Testing SVD++

In [6]:
import pandas as pd
from surprise import Reader, Dataset, SVDpp, accuracy
from surprise.model_selection import train_test_split
import timeit

# Timer start
start_time = timeit.default_timer()

# Prepare Data
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(train_df[['userId', 'movieId', 'rating']], reader)

# Sample 30% of the data for training
sample_data = train_df.sample(frac=0.2, random_state=42)
sample_data = Dataset.load_from_df(sample_data[['userId', 'movieId', 'rating']], reader)
trainset = sample_data.build_full_trainset()

# Train the SVD++ model with predefined parameters on the sampled data
algo = SVDpp()
algo.fit(trainset)

# Prepare test set
testset = list(zip(test_df['userId'], test_df['movieId'], [0]*len(test_df)))
predictions = algo.test(testset)
predicted_ratings = [pred.est for pred in predictions]

# Add predicted ratings to test_df
test_df['rating'] = predicted_ratings
test_df['ID'] = test_df.apply(lambda row: f"{int(row['userId'])}_{int(row['movieId'])}", axis=1)
result_df = test_df[['ID', 'rating']]

# Save the results to a CSV file
result_df.to_csv('SVDpp.csv', index=False)
print(result_df.shape)
print(result_df.head())

# Timing the amount of time it takes to complete the process
elapsed = timeit.default_timer() - start_time
print(f"{elapsed} seconds or {elapsed / 60} minutes")

# Calculate and print RMSE
test_rmse = accuracy.rmse(predictions)
print(f"SVD++ Model RMSE: {test_rmse}")

(5000019, 2)
       ID    rating
0  1_2011  3.738186
1  1_4144  4.059203
2  1_5767  3.275729
3  1_6711  3.561474
4  1_7318  3.204849
1050.5379753010002 seconds or 17.50896625501667 minutes
RMSE: 3.5929
SVD++ Model RMSE: 3.5929230983854774


In [ ]:
# This was build with fraction of 0.1 with custom parameters
#        ID    rating
# 0  1_2011  3.760092
# 1  1_4144  4.003556
# 2  1_5767  3.545112
# 3  1_6711  4.325373
# 4  1_7318  3.069776
# 900.9963529969991 seconds or 15.016605883283319 minutes
# RMSE: 3.5714
# SVD++ Model RMSE: 3.571350377637289

In [7]:
# This is with SVD no-params
#        ID    rating
# 0  1_2011  3.738186
# 1  1_4144  4.059203
# 2  1_5767  3.275729
# 3  1_6711  3.561474
# 4  1_7318  3.204849
# 1050.5379753010002 seconds or 17.50896625501667 minutes
# RMSE: 3.5929
# SVD++ Model RMSE: 3.5929230983854774

SyntaxError: invalid syntax (940679574.py, line 1)

In [ ]:
import pandas as pd
from surprise import Reader, Dataset, NMF, accuracy
from surprise.model_selection import train_test_split
import timeit

# Timer start
start_time = timeit.default_timer()

# Prepare Data
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(train_df[['userId', 'movieId', 'rating']], reader)

# Sample 30% of the data for training
# sample_data = train_df.sample(frac=1, random_state=42)
# sample_data = Dataset.load_from_df(data[['userId', 'movieId', 'rating']], reader)
trainset = data.build_full_trainset()

# Define the NMF algorithm
algo = SlopeOne()

# Train the NMF model with predefined parameters on the sampled data
algo.fit(trainset)

# Prepare test set
testset = list(zip(test_df['userId'], test_df['movieId'], [0]*len(test_df)))
predictions = algo.test(testset)
predicted_ratings = [pred.est for pred in predictions]

# Add predicted ratings to test_df
test_df['rating'] = predicted_ratings
test_df['ID'] = test_df.apply(lambda row: f"{int(row['userId'])}_{int(row['movieId'])}", axis=1)
result_df = test_df[['ID', 'rating']]

# Save the results to a CSV file
result_df.to_csv('SlopeOne.csv', index=False)
print(result_df.shape)
print(result_df.head())

# Timing the amount of time it takes to complete the process
elapsed = timeit.default_timer() - start_time
print(f"{elapsed} seconds or {elapsed / 60} minutes")

# Calculate and print RMSE
test_rmse = accuracy.rmse(predictions)
print(f"NMF Model RMSE: {test_rmse}")